In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os, time
import random
import cv2

In [3]:
train_data = np.load(r"train_data.npy")
val_data = np.load(r"val_data.npy")
test_data = np.load(r"test_data.npy")